# pyBumpHunter : A model agnostic bump hunting tool in python

**presenter : Louis VASLIN  -  Université Clermont Auvergne (FR)**

![pyBH_logo](img/pyBH_logo.png)


[Link to github](https://github.com/scikit-hep/pyBumpHunter)  
[Link to PyPI](https://pypi.org/project/pyBumpHunter/)

In [ ]:
import numpy as np
import pyBumpHunter as BH
import matplotlib.pyplot as plt
from matplotlib import colors as mcl

## The BumpHunter algorithm

BumpHunter is a usefull algorithm used in HEP community that allows to look for a **excess** (or deficit) in a **data** distribution w.r.t. a **reference background**.

It will compute the **local** and **global p-value** of the most significant **localized deviation** (bump) in the data.

The global p-value is computed by **generating toys** from the reference and scaning them like real data.  
The global p-value is then *the p-value of the local p-values*.

### Why pyBumpHunter ?

Public implementation based on pure python (numpy/scipy).  
Doesn't depend on the ROOT software.  
Recently integrated to Scikit-HEP.  
Still under active development.

## 1D scan

The basic scan that compare a data distribution to a refference background.  
It will give the position and width of the bump, as well as its local and global p-value.  
Also give a *rought* evaluation of the signal content of the bump (data - ref).

In [ ]:
# Generate some background and data
np.random.seed(42)
bkg = np.random.exponential(scale=8.0, size=500_000)
data = np.random.exponential(scale=8.0, size=500_000)
data = np.append(
    data,
    np.random.normal(loc=20, scale=3, size=1200)
)
rng = [0,70]

# Make histograms
bkg_hist, bins = np.histogram(bkg, bins=50, range=rng)
data_hist, _ = np.histogram(data, bins=bins)

# Plot the distributions
F = plt.figure(figsize=(12,8))
plt.title('Example distributions', size='xx-large')
plt.hist(bkg, bins=bins, histtype='step', lw=2, label='bkg')
plt.hist(data, bins=bins, histtype='step', lw=2, label='data')
plt.legend(fontsize='x-large')
plt.xticks(fontsize='x-large')
plt.yticks(fontsize='x-large')
plt.yscale('log')
plt.show()

In [ ]:
# Create a BumpHunter1D instance
bh1 = BH.BumpHunter1D(
    width_min=2,
    width_max=6,
    width_step=1,
    scan_step=1,
    bins=bins,
    rang=rng,
    npe=10_000,
    nworker=1,
    seed=666
)

# Call the scan method
# Here we give the data as non-histogramed distribution
bh1.bump_scan(data, bkg)

# Display results
print('##.print_bump_info() call##')
bh1.print_bump_info()
print('##.print_bump_true() call##')
bh1.print_bump_true(data, bkg)

bh1.plot_bump(data, bkg)
bh1.plot_tomography(data)
bh1.plot_stat(show_Pval=True)

* **NOTE : The following code will give the same results**
```python
# The data is given as histogramed distribution
# In this case, we must specify `is_hist=True` in the argument
bh1.bump_scan(data_hist, bkg_hist, is_hist=True)
bh1.print_bump_info()
bh1.print_bump_true(data_hist, bkg_hist, is_hist)
bh1.plot_bump(data_hist, bkg_hist, is_hist=True)
bh1.plot_tomography(data_hist, bkg_hist, is_hist=True)
bh1.plot_stat(show_Pval=True)
```

## Access to results

The results of the previous scans is stored in the BumpHunter1D object.  
You can easily access them to make your own tests and/or plot routines.

In [ ]:
print(f'local p-value of the data : {bh1.min_Pval_ar[0]:.5g}')
print(f'width of the bump (in number of bins) : {bh1.min_width_ar[0]}')
print(f'global significance : {bh1.significance:.5f}')

## Signal injection

Add the possibility to perform signal injection test using the BumpHunter algorithm to evaluate the sensibility to a given signal model.  
The injection is perform based on the reference background distribution and a signal distribution.  
The injections stops when the required sensitivity is reached (based on global significance).  
Currently, signal injection is only available in 1D.

In [ ]:
# Generate some signal
sig = np.random.normal(loc=20, scale=3, size=10_000)
sig_hist, _  = np.histogram(sig, bins=bins)

# Plot the signal distribution
F = plt.figure(figsize=(12,8))
plt.title('Signal distribution', size='xx-large')
plt.hist(sig, bins=bins, histtype='step', lw=2, label='signal')
plt.legend(fontsize='x-large')
plt.xticks(fontsize='x-large')
plt.yticks(fontsize='x-large')
plt.show()

In [ ]:
# Set the expected number of signal event
# This will be used to normalize the signal and compute the signal strength
# Here, we expect to have 500 signal events for a signal strength of 1.0
bh1.signal_exp = 1000

# Set the injection parameters
bh1.str_min = -1
bh1.str_scale = 'log'
bh1.sigma_limit = 3 # Injection stops when we reach 3 sigma global

# Call the injection method
bh1.signal_inject(sig_hist, bkg_hist, is_hist=True)

# Display results
bh1.plot_inject()

## 2D scan

This is the extension of the BumpHunter algorithm to 2D distributions.  
Works the same way as in 1D.

In [ ]:
# Generate some background and data
np.random.seed(42)
bkg = np.random.exponential(scale=8.0, size=(500_000,2))
data = np.random.exponential(scale=8.0, size=(500_000,2))
data = np.append(
    data,
    np.random.normal(loc=20, scale=3, size=(1100,2)),
    axis=0
)
rng = [[0,40], [0,40]]

# Make histograms
bkg_hist, binx, biny = np.histogram2d(bkg[:,0],bkg[:,1], bins=20, range=rng)
data_hist, _, _ = np.histogram2d(data[:,0], data[:,1], bins=[binx,biny])

# Plot the data distribution
F = plt.figure(figsize=(12,10))
plt.title('Data 2D distribution', size='xx-large')
plt.hist2d(data[:,0], data[:,1], bins=[binx,biny], norm=mcl.LogNorm())
plt.colorbar()
plt.xticks(fontsize='x-large')
plt.yticks(fontsize='x-large')
plt.show()

In [ ]:
# Create a BumpHunter2D instance
bh2 = BH.BumpHunter2D(
    width_min=[2,2],
    width_max=[3,3],
    width_step=[1,1],
    scan_step=[1,1],
    bins=[binx,biny],
    rang=rng,
    npe=6_000,
    nworker=1,
    seed=666
)

# Call the scan method
bh2.bump_scan(data_hist, bkg_hist, is_hist=True)

# Display results
# NOTE : There is no tomography plot in 2D
print('##.print_bump_info() call##')
bh2.print_bump_info()
print('##.print_bump_true() call##')
bh2.print_bump_true(data_hist, bkg_hist, is_hist=True)

bh2.plot_bump(data_hist, bkg_hist, is_hist=True)
bh2.plot_stat(show_Pval=True)

## Side-band normalization

This option allows to correct the number of background events in the calculation of the local p-value.  
The normalization scale is computed as the data/background ratio outside of the bump window.  
![rescale](img/rescale.png) 
This normalization is done automatically durring the scan.  
This metohd can also penalize the p-value when there is a discrepency between data and background ouside of the bump window.

In [ ]:
# Generating some background and data
# Generate some background and data
np.random.seed(44)
bkg = np.random.exponential(scale=8.0, size=800_000)
data = np.random.exponential(scale=8.0, size=500_000)
data = np.append(
    data,
    np.random.normal(loc=20, scale=3, size=1200)
)
rng = [0,70]

# Make histograms
bkg_hist, bins = np.histogram(bkg, bins=50, range=rng)
data_hist, _ = np.histogram(data, bins=bins)
bkg_hist = np.array(bkg_hist, dtype=float)


# Plot the distributions
F = plt.figure(figsize=(12,8))
plt.title('Example distributions', size='xx-large')
plt.hist(bkg, bins=bins, histtype='step', lw=2, label='bkg')
plt.hist(data, bins=bins, histtype='step', lw=2, label='data')
plt.legend(fontsize='x-large')
plt.xticks(fontsize='x-large')
plt.yticks(fontsize='x-large')
plt.yscale('log')
plt.show()

In [ ]:
# Create a BumpHunter1D instance
bh1 = BH.BumpHunter1D(
    width_min=2,
    width_max=6,
    width_step=1,
    scan_step=1,
    bins=bins,
    rang=rng,
    npe=10_000,
    nworker=1,
    use_sideband=True,
    seed=666
)

# Call the scan method
# Here we give the data as non-histogramed distribution
bh1.bump_scan(data_hist, bkg_hist, is_hist=True)

# Display results
print('##.print_bump_info() call##')
bh1.print_bump_info()
print('##.print_bump_true() call##')
bh1.print_bump_true(data_hist, bkg_hist, is_hist=True)

bh1.plot_bump(data_hist, bkg_hist, is_hist=True)
bh1.plot_tomography(data_hist, is_hist=True)
bh1.plot_stat(show_Pval=True)

## Multi-channel combination (under development)

This option allows to combine multiple channels in order to obtain a combined global p-value and significance.  
There are two combination method :

* **multiply**  
    Applyable only if all channels have the same binning.  
    For each position and width, the combined local p-value is defined as the product of the local p-value per channel.  
    The bump window is then defined as usual.

* **exclude**  
    Applyable even if all channels have different binning.  
    The combined bump window is defined as the intersection of the bump windows of each channel.  
    If there is no intersection, then the local p-value is set to 1.  
    Otherwise, the combined local p-value is the product of the local p-value per channel.

### Planned syntax

To define multiple channels with different bining, we pass a *list of bining* to the `bins` option.

```python
bh = BH.BumpHunter1D()
bh.bins=[40,50] # 40 bins for ch0 and 50 bins for ch1
```

To call a scan method in one of the two multi-channel modes, we pass a list of data and background distributions in argument.  
The choice of the combination method is done with the `comb` argument.
```python
bh.bump_scan(data, bkg, comb='exclude')
```

## Plan and future development

### For the next release

* Adding multi-channel combination

### Major additions planned

* API refactoring, with a DataHandler class that manage the histograming part separatly

* Add possibility to treat systematic uncertainties

* Other features that might be interesting (***your* ideas are welcome!**)